In [ ]:
import glob, os

import numpy as np
import pandas as pd

from StaticThreshold import event_search

import matplotlib.pyplot as plt
import seaborn as sns

### MATLAB Python Utils

In [ ]:
import scipy.io as spio
def formatData(t,s):
    if not isinstance(t,dict) and not isinstance(t,list):
        print("\t"*s+str(t))
    else:
        for key in t:
            print("\t"*s+str(key))
            if not isinstance(t,list):
                formatData(t[key],s+1)

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)


def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
        elif isinstance(dict[key], np.ndarray):
            dict[key] = _check_list(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        elif isinstance(elem, np.ndarray):
            dict[strg] = _check_list(elem)
        else:
            dict[strg] = elem
    return dict

def _check_list(list_obj):
    new_list = []
    for el in list_obj:
        if isinstance(el, spio.matlab.mio5_params.mat_struct):
            new_list.append(_todict(el))
        else:
            new_list.append(el)
    return new_list

## Load in Data

In [ ]:
setpoint = 0.1 # Setpoint of Interest

In [ ]:
import glob
import os

directory = 'INSERT_DIRECTORY' # Path to directory containing subdirectories of ECSTM data

folders = glob.glob(os.path.join(directory, '*ECSTM*'))

folders

In [ ]:
all_data = []
# For each subdirectory
for folder in folders:
    # Load data from mat files generated by ItSaveData.m
    file = glob.glob(os.path.join(folder, 'I(t) Spec/*{}*.mat'.format(setpoint)))
    data = loadmat(file[0])
    traces = []
    for trace in data['importedfiles']:
        traces.append(np.array(trace['structdata']))
    all_data.append(traces)
    print(folder)


## Event Search

In [ ]:
evts_all = []
for file in all_data:
    raw_evts = event_search(np.array(file), file, 3, 0)
    evts_all.append(raw_evts)     

## Event Analysis

Filter out events whose heights or durations are not desired

In [ ]:
def event_filter(dur, height, dur_lims, height_lims):
    return (dur > dur_lims[0]) and (dur < dur_lims[1]) and (height > height_lims[0]) and (height < height_lims[1])

dur_lims = (0, 1000) # Duration MinMax Limits
height_lims = (0, 1000) # Height MinMax Limits

evt_lens_all = []
evt_heights_all = []
for i in range(len(evts_all)):
    evt_lens = []
    evt_heights = []
    for j in range(len(evts_all[i])):
        trace = all_data[i][j]
        trace_evts = evts_all[i][j]
        for evt in trace_evts:
            dur = evt[1] - evt[0]
            height = trace[evt[0] : evt[1]].mean() - trace.mean()
            
            if event_filter(dur, height, dur_lims, height_lims):
                evt_lens.append(dur)
                evt_heights.append(height)
            
    evt_lens_all.append(evt_lens)
    evt_heights_all.append(evt_heights)
    

#### Event duration vs Event height scatter

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
for i in range(len(folders)):
    ax.scatter(evt_lens_all[i], evt_heights_all[i], s=1)

#### Event duration distributions

In [ ]:
bin_edges = np.histogram_bin_edges(evt_lens_all[0], bins=171)
bin_mids = bin_edges[1:] - ((bin_edges[1] - bin_edges[0]) / 2)

fig, ax = plt.subplots()
counts = np.histogram(evt_lens_all[3], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

counts = np.histogram(evt_lens_all[2], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

counts = np.histogram(evt_lens_all[1], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

counts = np.histogram(evt_lens_all[0], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

counts = np.histogram(evt_lens_all[4], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

#ax.set(ylim=(0, 5000))

#### Event height distributions

In [ ]:
bin_edges = np.histogram_bin_edges(evt_heights_all[0], bins=171)
bin_mids = bin_edges[1:] - ((bin_edges[1] - bin_edges[0]) / 2)

fig, ax = plt.subplots()
counts = np.histogram(evt_heights_all[3], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

counts = np.histogram(evt_heights_all[2], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

counts = np.histogram(evt_heights_all[1], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

counts = np.histogram(evt_heights_all[0], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

counts = np.histogram(evt_heights_all[4], bins=bin_edges)[0]
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges), alpha=0.7)

#### Display example event

In [ ]:
%matplotlib qt
file=3
i = 23
y = all_data[file][i]
x = np.linspace(0, len(y), len(y))

fig, ax = plt.subplots()
ax.plot(x, y)
for evt in evts_all[file][i]:
    ax.plot(x[evt[0] : evt[1]], y[evt[0] : evt[1]], color='red')

# All Setpoints

## Load Data

In [ ]:
setpoints = [0.01, 0.1, 1.0, 2.0, 3.0] # Setpoints of Interest
directory = 'INSERT_DIRECTORY' # Path to directory containing subdirectories of ECSTM data

folders = glob.glob(os.path.join(directory, '*ECSTM*'))
folders

In [ ]:
all_data_setpoints = []
# For each setpoint
for setpoint in setpoints:
    print("Setpoint: {}".format(setpoint))
    all_data = []
    # For each subdirectory
    for folder in folders:
        print(folder)
        # Load data from mat files generated by ItSaveData.m
        file = glob.glob(os.path.join(folder, 'I(t) Spec/*{}*.mat'.format(setpoint)))
        data = loadmat(file[0])
        traces = []
        for trace in data['importedfiles']:
            traces.append(np.array(trace['structdata']))
        all_data.append(traces)
        
    all_data_setpoints.append(all_data)
    

## Event Search

In [ ]:
evts_setpoints_all = []
for setpoint in all_data_setpoints:
    evts_all = []
    for file in setpoint:
        raw_evts = event_search(np.array(file), file, 3, 0)
        evts_all.append(raw_evts)
    evts_setpoints_all.append(evts_all)

## Event Analysis

Filter out events with undesirable heights and durations

In [ ]:
def event_filter(dur, height, dur_lims, height_lims):
    return (dur > dur_lims[0]) and (dur < dur_lims[1]) and (height > height_lims[0]) and (height < height_lims[1])

dur_lims = (0, 4E-3)
height_lims = (0, 2)

evt_lens_setpoints_all = []
evt_heights_setpoints_all = []
for i in range(len(setpoints)):
    evt_lens_all = []
    evt_heights_all = []
    for j in range(len(evts_setpoints_all[i])):
        evt_lens = []
        evt_heights = []
        for k in range(len(evts_setpoints_all[i][j])):
            trace = all_data_setpoints[i][j][k]
            trace_evts = evts_setpoints_all[i][j][k]
            for evt in trace_evts:
                dur = (evt[1] - evt[0]) / 25000
                height = np.median(trace[evt[0] : evt[1]]) - setpoints[i]

                if event_filter(dur, height, dur_lims, height_lims):
                    evt_lens.append(dur)
                    evt_heights.append(height)

        evt_lens_all.append(evt_lens)
        evt_heights_all.append(evt_heights)
    evt_lens_setpoints_all.append(evt_lens_all)
    evt_heights_setpoints_all.append(evt_heights_all)
    

Preprocess all data for visualisation

In [ ]:
all_lens_flat = np.array(sum(sum(evt_lens_setpoints_all, []), [])) * 1000 # in ms
all_heights_flat = np.array(sum(sum(evt_heights_setpoints_all, []), []))

bin_edges_dur = np.histogram_bin_edges(all_lens_flat, bins=100)
bin_edges_height = np.histogram_bin_edges(all_heights_flat, bins=100)

bin_mids_dur = bin_edges_dur[1:] - ((bin_edges_dur[1] - bin_edges_dur[0]) / 2)
bin_mids_height = bin_edges_height[1:] - ((bin_edges_height[1] - bin_edges_height[0]) / 2)

#### Master plot of event properties
Columns correspond to event duration vs height, event duration histograms, and event height histograms respectively. Each row represents the different setpoints.

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(5, 3, sharex=False, sharey=False, figsize=(6, 7), dpi=600)

labels = ['(a)', '(b)', '(c)', '(d)', '(e)']

# plot graphs
for i in range(len(setpoints)):
    dur_norm = len(sum(evt_lens_setpoints_all[i], []))
    height_norm = len(sum(evt_heights_setpoints_all[i], []))
    for j in range(len(folders)):
        x = np.array(evt_lens_setpoints_all[i][j])
        ax[i, 0].scatter(x * 1000, evt_heights_setpoints_all[i][j], s=1)
        
        dur_counts = np.histogram(x * 1000, bins=bin_edges_dur)[0] / dur_norm
        ax[i, 1].bar(bin_mids_dur, dur_counts, width=np.diff(bin_edges_dur), alpha=0.7)
        
        height_counts = np.histogram(evt_heights_setpoints_all[i][j], bins=bin_edges_height)[0] / height_norm 
        ax[i, 2].bar(bin_mids_height, height_counts, width=np.diff(bin_edges_height), alpha=0.7)
        
# axis limits
for i in range(len(setpoints)):
    ax[i, 0].set(xlim=(0, 4), ylim=(0, 2))
    ax[i, 1].set(xlim=(0, 4), ylim=(0, 0.1))
    ax[i, 2].set(xlim=(0., 2), ylim=(0, 0.2))
    
    ax[i, 0].tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        ax[i, 0].spines[axis].set_linewidth(1.5)
    ax[i, 1].tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        ax[i, 1].spines[axis].set_linewidth(1.5)
    ax[i, 2].tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        ax[i, 2].spines[axis].set_linewidth(1.5)
    
    # remove axes
    if i != len(setpoints)-1:
        ax[i, 0].set(xticklabels=[])
        ax[i, 1].set(xticklabels=[])
        ax[i, 2].set(xticklabels=[])
    else:
        ax[i, 0].set_xlabel("Duration / ms", weight='bold')
        ax[i, 1].set_xlabel("Duration / ms", weight='bold')
        ax[i, 2].set_xlabel("Height / nm", weight='bold')
    
for i, a in enumerate(ax[:, 0]):
    a.set_ylabel("Height / nm", weight='bold')
    a.annotate(labels[i], xy=(0, 0), xytext=(-3, 2), weight='bold', va='center')
    
for a in ax[:, 1:]:
    for b in a:
        b.set_ylabel("#", weight='bold')

for a in ax[:, 1]:
    a.set(yticks=[0, 0.05, 0.1])
    
fig.tight_layout()

fig.savefig('EvtResSmall.png')

### Event frequency

In [ ]:
columns = ['Setpoint', 'Repeat', 'Frequency']
freq_df = pd.DataFrame(columns=columns)

for i in range(len(setpoints)):
    for j in range(len(folders)):
        freq = len(evt_lens_setpoints_all[i][j]) / 500
        new_row = pd.DataFrame([[setpoints[i], j, freq]], columns=columns)
        freq_df = pd.concat([freq_df, new_row], ignore_index=True)
freq_df       
    

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=600)
sns.barplot(data=freq_df, x='Setpoint', y='Frequency', ax=ax)

ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)
    
ax.set_xlabel('Setpoint / nA', weight='bold')
ax.set_ylabel('Frequency / s\N{SUPERSCRIPT MINUS}\N{SUPERSCRIPT ONE}', weight='bold')

fig.savefig("EvtFreq.png")

### Additional Filtering

In [ ]:
def event_filter(dur, height, dur_lims, height_lims):
    return (dur > dur_lims[0]) and (dur < dur_lims[1]) and (height > height_lims[0]) and (height < height_lims[1])

dur_lims = (0.0005, 1.5E-3) # Duration MinMax Limits
height_lims = (0.4, 2) # Height MinMax Limits

evt_lens_setpoints_all = []
evt_heights_setpoints_all = []
all_setpoint_passed = []
for i in range(len(setpoints)):
    evt_lens_all = []
    evt_heights_all = []
    setpoint_passed = []
    for j in range(len(evts_setpoints_all[i])):
        evt_lens = []
        evt_heights = []
        file_passed = []
        for k in range(len(evts_setpoints_all[i][j])):
            trace = all_data_setpoints[i][j][k]
            trace_evts = evts_setpoints_all[i][j][k]
            trace_passed = []
            for evt in trace_evts:
                dur = (evt[1] - evt[0]) / 25000
                height = trace[evt[0] : evt[1]].mean() - trace.mean()
                passed = 0
                if event_filter(dur, height, dur_lims, height_lims):
                    passed = 1
                    evt_lens.append(dur)
                    evt_heights.append(height)
                trace_passed.append(passed)
            file_passed.append(trace_passed)
        setpoint_passed.append(file_passed)            
        evt_lens_all.append(evt_lens)
        evt_heights_all.append(evt_heights)
    all_setpoint_passed.append(setpoint_passed)    
    evt_lens_setpoints_all.append(evt_lens_all)
    evt_heights_setpoints_all.append(evt_heights_all)

### Plot Example Events

In [ ]:
%matplotlib qt
setpoint_idx = 1
repeat = 0

i = 26
y = all_data_setpoints[setpoint_idx][repeat][i]
x = np.linspace(0, len(y), len(y))

fig, ax = plt.subplots()
ax.plot(x, y)
for j, evt in enumerate(evts_setpoints_all[setpoint_idx][repeat][i]):
    #if all_setpoint_passed[setpoint_idx][repeat][i][j] == 1:
        ax.plot(x[evt[0] : evt[1]], y[evt[0] : evt[1]], color='red')

In [ ]:
%matplotlib qt
setpoint_idx = 1
repeat = 3

i = 23
y = all_data_setpoints[setpoint_idx][repeat][i]
x = np.linspace(0, len(y), len(y))

fig, ax = plt.subplots()
ax.plot(x, y)
for j, evt in enumerate(evts_setpoints_all[setpoint_idx][repeat][i]):
    #if all_setpoint_passed[setpoint_idx][repeat][i][j] == 1:
        ax.plot(x[evt[0] : evt[1]], y[evt[0] : evt[1]], color='red')

### Comparison of believed true positives vs false positives

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=600, sharey=True)

sp1 = 1
rep1 = 3
idx1 = 23
y1 = all_data_setpoints[sp1][rep1][idx1]
x1 = np.linspace(0, len(y1), len(y1)) / 25000

sp2 = 1
rep2 = 0
idx2 = 26
y2 = all_data_setpoints[sp2][rep2][idx2]
x2 = np.linspace(0, len(y2), len(y2)) / 25000

axs[1].plot(x1, y1)
for j, evt in enumerate(evts_setpoints_all[sp1][rep1][idx1]):
    axs[1].plot(x1[evt[0] : evt[1]], y1[evt[0] : evt[1]], color='red')
    
axs[0].plot(x2, y2)
for j, evt in enumerate(evts_setpoints_all[sp2][rep2][idx2]):
    axs[0].plot(x2[evt[0] : evt[1]], y2[evt[0] : evt[1]], color='red')
    
axs[1].set(xlim=(0.19, 0.22))
axs[1].set_xlabel('Time / s', weight='bold')
#axs[1].set_ylabel('Current / nA', weight='bold')

axs[0].set(xlim=(0.33, 0.36))
axs[0].set_ylabel('Current / nA', weight='bold')
axs[0].set_xlabel('Time / s', weight='bold')

axs[1].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[1].spines[axis].set_linewidth(1.5)
axs[0].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[0].spines[axis].set_linewidth(1.5)
    
axs[1].annotate('(b)', xy=(0.19, 0.1), xytext=(0.185, 0.185), weight='bold')
axs[0].annotate('(a)', xy=(0.33, 0.1), xytext=(0.320, 0.185), weight='bold')

fig.tight_layout()

# fig.savefig('ExampleEvts.png')